In [1]:
import random
import numpy as np
import tensorflow as tf
import os
import json
from zipfile import ZipFile
from PIL import Image
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras import layers, models
import cv2
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import roc_curve, auc
import seaborn as sns
import pandas as pd

# Set seeds for reproducibility
random.seed(0)
np.random.seed(0)
tf.random.set_seed(0)

# Enable Apple Metal GPU acceleration
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e:
        print(e)

# Image Parameters
img_size = 224  # Target image size
batch_size = 32  # Batch size for model training

# Dataset Path
base_dir = '/Users/bhaveshreddy/Desktop/Capstone Project/plantvillage dataset'

# Check if the dataset folders exist
print("Dataset Folders:", os.listdir(base_dir))

# Image Data Generators
data_gen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2  # Use 20% of data for validation
)

# Train Generator
train_generator = data_gen.flow_from_directory(
    os.path.join(base_dir, "color"),
    target_size=(img_size, img_size),
    batch_size=batch_size,
    subset='training',
    class_mode='categorical'
)

# Validation Generator
validation_generator = data_gen.flow_from_directory(
    os.path.join(base_dir, "color"),
    target_size=(img_size, img_size),
    batch_size=batch_size,
    subset='validation',
    class_mode='categorical'
)

Dataset Folders: ['grayscale', '.DS_Store', 'segmented', 'color']
Found 43456 images belonging to 38 classes.
Found 10849 images belonging to 38 classes.


In [2]:
# Load Pretrained MobileNetV2 Model from Local File
weights_path = '/Users/bhaveshreddy/Downloads/mobilenet_v2_weights_tf_dim_ordering_tf_kernels_1.0_224_no_top.h5'
base_model = MobileNetV2(weights=weights_path, include_top=False, input_shape=(img_size, img_size, 3))
base_model.trainable = False  # Freeze the base model layers

# Build the Model
model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.3),
    layers.Dense(len(train_generator.class_indices), activation='softmax')
])

# Compile the Model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the Model
model.fit(
    train_generator,
    validation_data=validation_generator,
    epochs=10,
    callbacks=[EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)]
)


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
1358/1358 ━━━━━━━━━━━━━━━━━━━━ 1983s 1s/step - accuracy: 0.7197 - loss: 1.0323 - val_accuracy: 0.9307 - val_loss: 0.2208
Epoch 2/10
1358/1358 ━━━━━━━━━━━━━━━━━━━━ 1964s 1s/step - accuracy: 0.9136 - loss: 0.2676 - val_accuracy: 0.9434 - val_loss: 0.1676
Epoch 3/10
1358/1358 ━━━━━━━━━━━━━━━━━━━━ 1983s 1s/step - accuracy: 0.9317 - loss: 0.2087 - val_accuracy: 0.9465 - val_loss: 0.1562
Epoch 4/10
1358/1358 ━━━━━━━━━━━━━━━━━━━━ 2083s 2s/step - accuracy: 0.9393 - loss: 0.1767 - val_accuracy: 0.9471 - val_loss: 0.1544
Epoch 5/10
1358/1358 ━━━━━━━━━━━━━━━━━━━━ 1968s 1s/step - accuracy: 0.9435 - loss: 0.1598 - val_accuracy: 0.9518 - val_loss: 0.1400
Epoch 6/10
1358/1358 ━━━━━━━━━━━━━━━━━━━━ 2017s 1s/step - accuracy: 0.9523 - loss: 0.1370 - val_accuracy: 0.9532 - val_loss: 0.1345
Epoch 7/10
1358/1358 ━━━━━━━━━━━━━━━━━━━━ 2130s 2s/step - accuracy: 0.9534 - loss: 0.1294 - val_accuracy: 0.9558 - val_loss: 0.1302
Epoch 8/10
1358/1358 ━━━━━━━━━━━━━━━━━━━━ 1889s 1s/step - accuracy: 0.9573 -

In [6]:
def load_and_preprocess_image(image_path, target_size=(224, 224)):
    img = Image.open(image_path).convert('RGB')
    img = img.resize(target_size, Image.Resampling.LANCZOS)
    img_array = np.array(img).astype('float32') / 255.0
    img_array = np.expand_dims(img_array, axis=0)
    return img_array

# Function to Predict the Class of an Image
def predict_image_class(model, image_path, class_indices):
    preprocessed_img = load_and_preprocess_image(image_path)
    predictions = model.predict(preprocessed_img)
    predicted_class_index = np.argmax(predictions, axis=1)[0]
    class_indices = {v: k for k, v in class_indices.items()}  # Reverse mapping
    predicted_class_name = class_indices.get(predicted_class_index, "Unknown")
    return predicted_class_name

# Save class indices to JSON
with open('class_indices.json', 'w') as f:
    json.dump(train_generator.class_indices, f)

# Load class indices from JSON
with open('class_indices.json', 'r') as f:
    loaded_class_indices = json.load(f)

# Example Usage
image_path = '/Users/bhaveshreddy/Desktop/cardio-risk-prediction/checking.JPG'
predicted_class_name = predict_image_class(model, image_path, loaded_class_indices)
print("Predicted Class Name:", predicted_class_name)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
Predicted Class Name: Apple___Black_rot


In [7]:
# Save the Model
model.save('MobileNetV2-2.h5')
